In [1]:
import ftplib
import os
import pandas as pd

ftp://ftp.datasus.gov.br/dissemin/publicos/SIM/CID10/DORES


DOSP2023.dbc

In [2]:
# --- Configurações do FTP ---
FTP_HOST = "ftp.datasus.gov.br"  # Substitua pelo seu host
FTP_DIR = "/dissemin/publicos/SIM/CID10/DORES/" # O diretório onde o arquivo está
FILENAME_TO_DOWNLOAD = "DOSP2023.dbc"         # O arquivo que queremos baixar
DOWNLOAD_DIR = "dados/" # Pasta local para salvar os arquivos

In [3]:
os.makedirs(DOWNLOAD_DIR, exist_ok=True)
local_filepath = os.path.join(DOWNLOAD_DIR, FILENAME_TO_DOWNLOAD)

In [4]:
# --- Conexão ---
try:
    ftp = ftplib.FTP(FTP_HOST)
    ftp.login() # Login anônimo
    print(f"Login anônimo no {FTP_HOST} realizado com sucesso.")
    
    # --- MUDANÇA NA LÓGICA ---
    
    # 1. Entra no diretório CORRETO
    ftp.cwd(FTP_DIR)
    print(f"Navegado para o diretório: {FTP_DIR}")

    # 2. Não precisamos listar arquivos (ftp.nlst())
    # 3. Não precisamos de loop (for ...:)
    
    # 4. Baixa o arquivo específico diretamente
    print(f"Baixando {FILENAME_TO_DOWNLOAD} para {local_filepath}...")
    
    # Abre o arquivo local em modo 'wb' (write binary)
    with open(local_filepath, 'wb') as local_file:
        # Comando para baixar o arquivo
        ftp.retrbinary(f'RETR {FILENAME_TO_DOWNLOAD}', local_file.write)
    
    print(f"Download de {FILENAME_TO_DOWNLOAD} concluído.")

    # Fecha a conexão
    ftp.quit()
    print("Conexão FTP fechada.")

except ftplib.all_errors as e:
    print(f"Erro no FTP: {e}")

Login anônimo no ftp.datasus.gov.br realizado com sucesso.
Navegado para o diretório: /dissemin/publicos/SIM/CID10/DORES/
Baixando DOSP2023.dbc para dados/DOSP2023.dbc...
Download de DOSP2023.dbc concluído.
Conexão FTP fechada.


In [ ]:
%pip install pysus

In [8]:
import os
import pandas as pd
# Importa a função de leitura do pysus
from pysus.utilities.readdbc import read_dbc

# --- Configuração dos Caminhos ---

# 1. Pega o caminho da pasta onde este script .py está
# (Use 'os.path.abspath(__file__)' se estiver executando como script)
# (Use 'os.getcwd()' se estiver em um notebook)
try:
    script_dir = os.path.dirname(os.path.abspath(__file__))
except NameError:
    # __file__ não é definido em ambientes interativos (como notebooks)
    # Nesses casos, usamos o diretório de trabalho atual
    script_dir = os.getcwd()
    print("Aviso: __file__ não definido. Usando diretório atual (cwd) como base.")


# 2. Define os diretórios com base na pasta do script
# O 'downloader_v2.py' salvou os dados em 'dados/'
DADOS_BRUTOS_DIR = os.path.join(script_dir, "dados")
# Onde salvaremos os CSVs processados
DADOS_PROCESSADOS_DIR = os.path.join(script_dir, "dados_processados")

# 3. Garante que a pasta de saída exista
os.makedirs(DADOS_PROCESSADOS_DIR, exist_ok=True)

# 4. Define os nomes dos arquivos
filename_dbc = "DOSP2023.dbc" # O arquivo que você baixou
filename_csv = "DOSP2023.csv" # O nome do arquivo final

# 5. Define os caminhos completos
path_dbc = os.path.join(DADOS_BRUTOS_DIR, filename_dbc)
path_csv = os.path.join(DADOS_PROCESSADOS_DIR, filename_csv)

print(f"Iniciando conversão de {filename_dbc}...")
print(f"Arquivo de entrada: {path_dbc}")
print(f"Arquivo de saída: {path_csv}")

try:
    # --- PASSO 1 e 2 (Combinados): Ler o .dbc direto para o Pandas ---
    
    # A função read_dbc faz tudo: descompacta e lê.
    # Usamos 'iso-8859-1' (latin1) porque é o encoding padrão do DATASUS.
    print(f"Lendo e descompactando {path_dbc}...")
    df = read_dbc(path_dbc, encoding='iso-8859-1')
    
    print(f"Arquivo lido com sucesso. Total de {len(df)} linhas.")
    print("\nPrimeiras 5 linhas dos dados:")
    print(df.head())

    # --- PASSO 3: Salvar como .csv ---
    print(f"\nSalvando como CSV em {path_csv}...")
    # Usamos index=False para não salvar o índice do Pandas no CSV
    # Usamos encoding 'utf-8' para garantir compatibilidade
    df.to_csv(path_csv, index=False, encoding='utf-8')
    print("Conversão para CSV concluída.")

except FileNotFoundError:
    print(f"ERRO: O arquivo de entrada não foi encontrado em {path_dbc}")
    print("Verifique se o script 'downloader_v2.py' foi executado com sucesso e o arquivo está na pasta 'dados'.")
except Exception as e:
    print(f"Ocorreu um erro inesperado: {e}")

Aviso: __file__ não definido. Usando diretório atual (cwd) como base.
Iniciando conversão de DOSP2023.dbc...
Arquivo de entrada: /home/carolina/lgcm/projects/data_science/Projeto/dados/DOSP2023.dbc
Arquivo de saída: /home/carolina/lgcm/projects/data_science/Projeto/dados_processados/DOSP2023.csv
Lendo e descompactando /home/carolina/lgcm/projects/data_science/Projeto/dados/DOSP2023.dbc...
Arquivo lido com sucesso. Total de 334303 linhas.

Primeiras 5 linhas dos dados:
  ORIGEM TIPOBITO   DTOBITO HORAOBITO NATURAL CODMUNNATU    DTNASC IDADE SEXO  \
0      1        2  01012023      0720     835     351492  08011945   477    2   
1      1        2  01012023      2300     822     220010  02091979   443    1   
2      1        2  01012023      2150     835     350400  17122022   215    1   
3      1        2  01072023      0815     841     410050  07121989   433    1   
4      1        2  01072023      1935     825     251660  06021942   481    2   

  RACACOR  ...  FONTES TPRESGINFO TPNIVE

In [1]:
import pandas as pd
import os

In [ ]:
csv_path = "/Projeto/dados_processados/DOSP2023.csv"

In [3]:
print("\n--- Carregando Amostra (1000 linhas) ---")
df_amostra = pd.read_csv(csv_path, nrows=100)
df_amostra


--- Carregando Amostra (1000 linhas) ---


,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,FONTES,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR
0,1,2,1012023,720.0,835,351492,8011945,477,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
1,1,2,1012023,2300.0,822,220010,2091979,443,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
2,1,2,1012023,2150.0,835,350400,17122022,215,1,1,...,SXXXXX,NaN,NaN,NaN,2022023.0,3.0,2022023.0,NaN,2.0,5
3,1,2,1072023,815.0,841,410050,7121989,433,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6
4,1,2,1072023,1935.0,825,251660,6021942,481,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,2,1012023,1502.0,835,355280,6062022,306,1,4,...,SXXXXX,NaN,NaN,NaN,11012023.0,3.0,NaN,NaN,1.0,285
96,1,2,1012023,725.0,835,355030,11011962,460,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,286
97,1,2,1012023,510.0,823,230450,17071943,479,2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,287
98,1,2,1012023,1340.0,835,350480,27031953,469,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,288


In [4]:
df_amostra.columns

Index(['ORIGEM', 'TIPOBITO', 'DTOBITO', 'HORAOBITO', 'NATURAL', 'CODMUNNATU',
       'DTNASC', 'IDADE', 'SEXO', 'RACACOR', 'ESTCIV', 'ESC', 'ESC2010',
       'SERIESCFAL', 'OCUP', 'CODMUNRES', 'LOCOCOR', 'CODESTAB', 'ESTABDESCR',
       'CODMUNOCOR', 'IDADEMAE', 'ESCMAE', 'ESCMAE2010', 'SERIESCMAE',
       'OCUPMAE', 'QTDFILVIVO', 'QTDFILMORT', 'GRAVIDEZ', 'SEMAGESTAC',
       'GESTACAO', 'PARTO', 'OBITOPARTO', 'PESO', 'TPMORTEOCO', 'OBITOGRAV',
       'OBITOPUERP', 'ASSISTMED', 'EXAME', 'CIRURGIA', 'NECROPSIA', 'LINHAA',
       'LINHAB', 'LINHAC', 'LINHAD', 'LINHAII', 'CAUSABAS', 'CB_PRE',
       'COMUNSVOIM', 'DTATESTADO', 'CIRCOBITO', 'ACIDTRAB', 'FONTE',
       'NUMEROLOTE', 'TPPOS', 'DTINVESTIG', 'CAUSABAS_O', 'DTCADASTRO',
       'ATESTANTE', 'STCODIFICA', 'CODIFICADO', 'VERSAOSIST', 'VERSAOSCB',
       'FONTEINV', 'DTRECEBIM', 'ATESTADO', 'DTRECORIGA', 'CAUSAMAT',
       'ESCMAEAGR1', 'ESCFALAGR1', 'STDOEPIDEM', 'STDONOVA', 'DIFDATA',
       'NUDIASOBCO', 'NUDIASOBIN', 'DTCADINV